In [1]:
from __future__ import print_function
import os
import sys
import math
import pickle
import boto3
import os
import numpy as np
import kg
import encoding
import pandas as pd
# from tqdm import tqdm
import time
import argparse
import logging
import re
# tqdm.pandas()
# pandarallel.initialize(progress_bar=True)
# bucket = os.environ.get("BUCKET_NAME", " ")
# raw_data_folder = os.environ.get("RAW_DATA", " ")
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# tqdm_notebook().pandas()
s3client = boto3.client('s3')

Using backend: pytorch


sys path is ['/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python36.zip', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/lib-dynload', '', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/dglke-0.1.0.dev0-py3.6.egg', '/home/ec2-user/workplace/Aws-gcr-sc-recommender-system/src/offline/docker/batch/container/batch/src/fasthan', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ec2-user/.ipython']


In [2]:
########################################
# 从s3同步数据
########################################
def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))


def write_to_s3(filename, bucket, key):
    print("upload s3://{}/{}".format(bucket, key))
    with open(filename, 'rb') as f:  # Read in binary mode
        # return s3client.upload_fileobj(f, bucket, key)
        return s3client.put_object(
            ACL='bucket-owner-full-control',
            Bucket=bucket,
            Key=key,
            Body=f
        )

def write_str_to_s3(content, bucket, key):
    print("write s3://{}/{}, content={}".format(bucket, key, content))
    s3client.put_object(Body=str(content).encode("utf8"), Bucket=bucket, Key=key, ACL='bucket-owner-full-control')

default_bucket = 'aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887'
default_prefix = 'sample-data'
parser = argparse.ArgumentParser()
parser.add_argument('--bucket', type=str, default=default_bucket)
parser.add_argument('--prefix', type=str, default=default_prefix)
args, _ = parser.parse_known_args()
bucket = args.bucket
prefix = args.prefix

print("bucket={}".format(bucket))
print("prefix='{}'".format(prefix))

out_s3_path = "s3://{}/{}/feature/content/inverted-list".format(bucket, prefix)

local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)

bucket=aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887
prefix='sample-data'


In [3]:
# prepare model for batch process
meta_file_prefix = "{}/model/meta_files".format(prefix)
os.environ['GRAPH_BUCKET'] = bucket
os.environ['KG_DBPEDIA_KEY'] = '{}/kg_dbpedia.txt'.format(meta_file_prefix)
os.environ['KG_ENTITY_KEY'] = '{}/entities_dbpedia.dict'.format(meta_file_prefix)
os.environ['KG_RELATION_KEY'] = '{}/relations_dbpedia.dict'.format(meta_file_prefix)
os.environ['KG_DBPEDIA_TRAIN_KEY'] = '{}/kg_dbpedia_train.txt'.format(meta_file_prefix)
os.environ['KG_ENTITY_TRAIN_KEY'] = '{}/entities_dbpedia_train.dict'.format(meta_file_prefix)
os.environ['KG_RELATION_TRAIN_KEY'] = '{}/relations_dbpedia_train.dict'.format(meta_file_prefix)
os.environ['KG_ENTITY_INDUSTRY_KEY'] = '{}/entity_industry.txt'.format(meta_file_prefix)
os.environ['KG_VOCAB_KEY'] = '{}/vocab.json'.format(meta_file_prefix)
os.environ['DATA_INPUT_KEY'] = ''
os.environ['TRAIN_OUTPUT_KEY'] = '{}/model/rank/content/dkn_embedding_latest/'.format(prefix)

kg_path = os.environ['GRAPH_BUCKET']
dbpedia_key = os.environ['KG_DBPEDIA_KEY']
entity_key = os.environ['KG_ENTITY_KEY']
relation_key = os.environ['KG_RELATION_KEY']
dbpedia_train_key = os.environ['KG_DBPEDIA_TRAIN_KEY']
entity_train_key = os.environ['KG_ENTITY_TRAIN_KEY']
relation_train_key = os.environ['KG_RELATION_TRAIN_KEY']
entity_industry_key = os.environ['KG_ENTITY_INDUSTRY_KEY']
vocab_key = os.environ['KG_VOCAB_KEY']
data_input_key = os.environ['DATA_INPUT_KEY']
train_output_key = os.environ['TRAIN_OUTPUT_KEY']

env = {
    'GRAPH_BUCKET': kg_path,
    'KG_DBPEDIA_KEY': dbpedia_key,
    'KG_ENTITY_KEY': entity_key,
    'KG_RELATION_KEY': relation_key,
    'KG_DBPEDIA_TRAIN_KEY': dbpedia_train_key,
    'KG_ENTITY_TRAIN_KEY': entity_train_key,
    'KG_RELATION_TRAIN_KEY': relation_train_key,
    'KG_ENTITY_INDUSTRY_KEY': entity_industry_key,
    'KG_VOCAB_KEY': vocab_key,
    'DATA_INPUT_KEY': data_input_key,
    'TRAIN_OUTPUT_KEY': train_output_key
}

print("Kg env: {}".format(env))
graph = kg.Kg(env)  # Where we keep the model when it's loaded
model = encoding.encoding(graph, env)

Kg env: {'GRAPH_BUCKET': 'aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887', 'KG_DBPEDIA_KEY': 'sample-data/model/meta_files/kg_dbpedia.txt', 'KG_ENTITY_KEY': 'sample-data/model/meta_files/entities_dbpedia.dict', 'KG_RELATION_KEY': 'sample-data/model/meta_files/relations_dbpedia.dict', 'KG_DBPEDIA_TRAIN_KEY': 'sample-data/model/meta_files/kg_dbpedia_train.txt', 'KG_ENTITY_TRAIN_KEY': 'sample-data/model/meta_files/entities_dbpedia_train.dict', 'KG_RELATION_TRAIN_KEY': 'sample-data/model/meta_files/relations_dbpedia_train.dict', 'KG_ENTITY_INDUSTRY_KEY': 'sample-data/model/meta_files/entity_industry.txt', 'KG_VOCAB_KEY': 'sample-data/model/meta_files/vocab.json', 'DATA_INPUT_KEY': '', 'TRAIN_OUTPUT_KEY': 'sample-data/model/rank/content/dkn_embedding_latest/'}
load file: aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/meta_files/kg_dbpedia_train.txt
load file: aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/meta_files/kg_dbpedia.txt
load file

In [4]:
graph.train(max_step=2000)

Reading train triples....
Finished. Read 8879 train triples.
|Train|: 8879
Total initialize time 0.030 seconds


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/dgl/base.py:25: UserWarning: multigraph will be deprecated.DGL will treat all graphs as multigraph in the future.
  warnings.warn(msg, warn_type)


[proc 0][Train](20/2000) average pos_loss: 6.734581112861633
[proc 0][Train](20/2000) average neg_loss: 2.103131738305092
[proc 0][Train](20/2000) average loss: 4.418856483697891
[proc 0][Train](20/2000) average regularization: 2.134832590172664e-05
[proc 0][Train] 20 steps take 0.996 seconds
[proc 0]sample: 0.057, forward: 0.473, backward: 0.420, update: 0.045
[proc 0][Train](40/2000) average pos_loss: 2.6433502078056335
[proc 0][Train](40/2000) average neg_loss: 1.2602474704384803
[proc 0][Train](40/2000) average loss: 1.9517988204956054
[proc 0][Train](40/2000) average regularization: 2.698947300814325e-05
[proc 0][Train] 20 steps take 0.872 seconds
[proc 0]sample: 0.041, forward: 0.397, backward: 0.389, update: 0.044
[proc 0][Train](60/2000) average pos_loss: 1.62146737575531
[proc 0][Train](60/2000) average neg_loss: 0.8824161246418953
[proc 0][Train](60/2000) average loss: 1.2519417464733125
[proc 0][Train](60/2000) average regularization: 2.7673388740367954e-05
[proc 0][Train] 2

[proc 0][Train](480/2000) average pos_loss: 0.5694218397140502
[proc 0][Train](480/2000) average neg_loss: 0.6739735722541809
[proc 0][Train](480/2000) average loss: 0.6216977074742317
[proc 0][Train](480/2000) average regularization: 5.2665136718133e-05
[proc 0][Train] 20 steps take 0.863 seconds
[proc 0]sample: 0.040, forward: 0.394, backward: 0.385, update: 0.043
[proc 0][Train](500/2000) average pos_loss: 0.5304099753499031
[proc 0][Train](500/2000) average neg_loss: 0.6625559940934181
[proc 0][Train](500/2000) average loss: 0.5964829891920089
[proc 0][Train](500/2000) average regularization: 5.176434879103908e-05
[proc 0][Train] 20 steps take 0.887 seconds
[proc 0]sample: 0.060, forward: 0.396, backward: 0.387, update: 0.043
[proc 0][Train](520/2000) average pos_loss: 0.5549931764602661
[proc 0][Train](520/2000) average neg_loss: 0.6730876848101616
[proc 0][Train](520/2000) average loss: 0.6140404298901558
[proc 0][Train](520/2000) average regularization: 5.1338978664716706e-05
[p

[proc 0][Train](940/2000) average pos_loss: 0.4152857482433319
[proc 0][Train](940/2000) average neg_loss: 0.5899156473577023
[proc 0][Train](940/2000) average loss: 0.5026006907224655
[proc 0][Train](940/2000) average regularization: 5.175144015083788e-05
[proc 0][Train] 20 steps take 0.862 seconds
[proc 0]sample: 0.040, forward: 0.393, backward: 0.385, update: 0.043
[proc 0][Train](960/2000) average pos_loss: 0.41851086765527723
[proc 0][Train](960/2000) average neg_loss: 0.5829063922166824
[proc 0][Train](960/2000) average loss: 0.5007086306810379
[proc 0][Train](960/2000) average regularization: 5.119121069583343e-05
[proc 0][Train] 20 steps take 0.854 seconds
[proc 0]sample: 0.039, forward: 0.391, backward: 0.383, update: 0.041
[proc 0][Train](980/2000) average pos_loss: 0.410799665749073
[proc 0][Train](980/2000) average neg_loss: 0.591093310713768
[proc 0][Train](980/2000) average loss: 0.5009464845061302
[proc 0][Train](980/2000) average regularization: 5.0853151515184435e-05
[

[proc 0][Train](1380/2000) average pos_loss: 0.3630969777703285
[proc 0][Train](1380/2000) average neg_loss: 0.563001025468111
[proc 0][Train](1380/2000) average loss: 0.46304899752140044
[proc 0][Train](1380/2000) average regularization: 5.1047898887190966e-05
[proc 0][Train] 20 steps take 0.860 seconds
[proc 0]sample: 0.055, forward: 0.387, backward: 0.379, update: 0.039
[proc 0][Train](1400/2000) average pos_loss: 0.37262808084487914
[proc 0][Train](1400/2000) average neg_loss: 0.5490254648029804
[proc 0][Train](1400/2000) average loss: 0.46082677245140075
[proc 0][Train](1400/2000) average regularization: 5.0801624820451255e-05
[proc 0][Train] 20 steps take 0.840 seconds
[proc 0]sample: 0.036, forward: 0.385, backward: 0.379, update: 0.038
[proc 0][Train](1420/2000) average pos_loss: 0.369179829955101
[proc 0][Train](1420/2000) average neg_loss: 0.5492779232561589
[proc 0][Train](1420/2000) average loss: 0.459228877723217
[proc 0][Train](1420/2000) average regularization: 5.1079696

[proc 0][Train](1820/2000) average pos_loss: 0.3450771734118462
[proc 0][Train](1820/2000) average neg_loss: 0.5163516908884048
[proc 0][Train](1820/2000) average loss: 0.43071443736553194
[proc 0][Train](1820/2000) average regularization: 5.1053774950560185e-05
[proc 0][Train] 20 steps take 0.858 seconds
[proc 0]sample: 0.039, forward: 0.392, backward: 0.384, update: 0.042
[proc 0][Train](1840/2000) average pos_loss: 0.3416964203119278
[proc 0][Train](1840/2000) average neg_loss: 0.51850825548172
[proc 0][Train](1840/2000) average loss: 0.43010233640670775
[proc 0][Train](1840/2000) average regularization: 5.110309048177442e-05
[proc 0][Train] 20 steps take 0.860 seconds
[proc 0]sample: 0.040, forward: 0.393, backward: 0.385, update: 0.042
[proc 0][Train](1860/2000) average pos_loss: 0.32558983862400054
[proc 0][Train](1860/2000) average neg_loss: 0.5309656962752343
[proc 0][Train](1860/2000) average loss: 0.42827776446938515
[proc 0][Train](1860/2000) average regularization: 5.109849

In [5]:
# !aws s3 cp s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/complete_dkn_word_embedding.npy
# !aws s3 cp s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_entity_embedding.npy s3://aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/complete_dkn_entity_embedding.npy

In [6]:
n = 0
for k, v in entities_dbpedia_train.items():
    print("k {} v {}".format(k,v))
    if n > 10:
        break
    n = n + 1

NameError: name 'entities_dbpedia_train' is not defined